In [35]:
import numpy as np
import scipy.stats

### Question 1.(c)

In [36]:
r = 0.01
theta = 0.08
kappa = 2
beta = 0.2
rho = -0.7
Y0 = 0.04
S0 = 50
T = 0.25
dt = 1 / (3*360)
M = 270

N = 4000  # simlulation number
K = 50

In [37]:
# Initialize S and Y
S = np.zeros((M+1, N))
Y = np.zeros((M+1, N))
S_tilde = np.zeros((M+1, N))
S[0] = S0
Y[0] = Y0
S_tilde[0] = S0

In [56]:
ξ = np.random.normal(loc=0, scale=1, size=(M, N))
η = np.random.normal(loc=0, scale=1, size=(M, N))

def func():
    ξ = np.random.normal(loc=0, scale=1, size=(M, N))
    η = np.random.normal(loc=0, scale=1, size=(M, N))
    for m in range(M):
        Y[m+1,:] = Y[m,:] + (theta-kappa*Y[m,:])*dt + beta*np.sqrt(Y[m,:]) * \
                   np.sqrt(dt)*(rho*ξ[m,:] + np.sqrt(1-rho**2)*η[m,:])
        S[m+1,:] = S[m,:] + S[m,:] * np.sqrt(Y[m,:])*np.sqrt(dt)*ξ[m,:] + S[m,:] * r*dt
    
        tmp1 = np.exp(-kappa*m*dt)-np.exp(-kappa*(m+1)*dt)
        tmp2 = (r-theta/2)*dt - 1/(2*kappa) * (Y0-theta)*(tmp1) + \
               np.sqrt(theta*dt + (Y0-theta)/kappa*tmp1) * np.sqrt(dt)*ξ[m,:]
        S_tilde[m+1,:] = S_tilde[m,:] * np.exp(tmp2)
    X = np.exp(-r*T) * np.maximum(S[M,:]-K, 0)
    X_tilde = np.exp(-r*T) * np.maximum(S_tilde[M,:]-K, 0)
    sigma = np.sqrt(theta + (1/(kappa*T)) * (Y0-theta) * (1-np.exp(-kappa* T)))
    X_mean = np.mean(X)
    print("X_mean:", X_mean)
    
    def BS(s, K, T, sigma, r):
        '''BS model for European call'''
        d1 = (np.log(s/K) -(r*sigma**2)*T/2) / (sigma*np.sqrt(T)) 
        d2 = d1 - sigma*np.sqrt(T)
        return s * scipy.stats.norm.cdf(d1) - np.exp(-r*T)*K*scipy.stats.norm.cdf(d2)
    
    EX_tilde = BS(S0, K, T, sigma, r)
    print("EX_tilde:", EX_tilde)
    b = np.sum((X - np.mean(X))*(X_tilde - EX_tilde)) / np.sum((X_tilde - EX_tilde)*(X_tilde - EX_tilde))
    price_contr = np.sum(X-b*(X_tilde-EX_tilde)) / N
    print("Price_control:", price_contr)
    return X_mean, price_contr
func()

X_mean: 2.041007681375717
EX_tilde: 2.249376242222059
Price_control: 2.080102749999979


(2.041007681375717, 2.080102749999979)

### Question 1.(d)

In [39]:
num_times = 5000
P = []
for i in range(num_times):
    EX = func()
    P.append(EX)
P.sort()

In [47]:
# variance of the two methods
CI_95 = (P[int(num_times*.25)], P[int(num_times*.975)])
CI_95

(2.0443125001023112, 2.1690757737837156)

In [49]:
# mean of the two methods
X_mean, price_contr = func()
X_mean, price_contr

EX_tilde: 2.249376242222059


(2.0547681303101615, 2.0982017954876975)

From the above we can know that the mean of the two methods are approximately the same (around 2.05),  
but the variance of control variate method is much smaller. 